# 问题

- 你想创建一个新的拥有一些额外功能的实例属性类型，比如类型检查。

## 解决方案

- 如果你想创建一个全新的实例属性，可以通过一个描述器类的形式来定义它的功
能。下面是一个例子：

In [1]:
class Integer:
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]
        
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('Expected an int')
        instance.__dict__[self.name] = value
        
    def __delete__(self, instance):
        del instance.__dict__[self.name]

- 一个描述器就是一个实现了三个核心的属性访问操作(get, set, delete) 的类，分别
为`__get__()` 、`__set__()` 和`__delete__()` 这三个特殊的方法。这些方法接受一个实
例作为输入，之后相应的操作实例底层的字典。

- 为了使用一个描述器，需将这个描述器的实例作为类属性放到一个类的定义中。例
如：

In [2]:
class Point:
    x = Integer('x')
    y = Integer('y')
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

- 当你这样做后，所有对描述器属性(比如x 或y) 的访问会被`__get__()` 、`__set__()`
和`__delete__()` 方法捕获到。例如：

In [4]:
p = Point(2, 3)
print(p.x)
print(p.y)

2
3


In [6]:
p.y = 5
print(p.y)

5


In [8]:
p.x = 2.3

TypeError: Expected an int

In [11]:
print(p.__dict__)
print(Integer.__dict__)

{'x': 2, 'y': 5}
{'__module__': '__main__', '__init__': <function Integer.__init__ at 0x000002A919D8D510>, '__get__': <function Integer.__get__ at 0x000002A919D8D730>, '__set__': <function Integer.__set__ at 0x000002A919D8D488>, '__delete__': <function Integer.__delete__ at 0x000002A919D8DD90>, '__dict__': <attribute '__dict__' of 'Integer' objects>, '__weakref__': <attribute '__weakref__' of 'Integer' objects>, '__doc__': None}


- 作为输入，描述器的每一个方法会接受一个操作实例。为了实现请求操作，会相应
的操作实例底层的字典(`__dict__` 属性)。描述器的self.name 属性存储了在实例字
典中被实际使用到的key。